In [5]:
from bordarank.datatsets.objectranking.synthetic_data_generator import make_intransitive_medoids
from sklearn.utils import check_random_state
from sklearn.model_selection import ShuffleSplit
from bordarank.general_ranking import GeneralObjectRanker
from bordarank.callbacks import DebugOutput
from bordarank.tuning import ParameterOptimizer
import logging
from bordarank.util import configure_tf_keras
from keras import backend as K

In [6]:
logging.basicConfig(filename="test_models.log", level=logging.DEBUG,
                        format='%(asctime)s %(name)s %(levelname)-8s %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger(name='Experiment')
configure_tf_keras(42)
tf_session = K.get_session()
logger.info('tf session configuration: {}'.format(repr(tf_session._config)))

In [ ]:
n_features = 2
n_instances = 10000
n_objects = 5
random_state = check_random_state(42)
skf = ShuffleSplit(n_splits=2, test_size=0.5, random_state=random_state)

ranker_params = {'n_objects': n_objects,'n_object_features':n_features}
fit_params = {'epochs': epochs,'log_callbacks':[DebugOutput()]}
medoids_params = {'n_instances': n_instances, 'n_features': n_features, 'n_objects': n_objects,
                      'random_state': random_state}
optimizer_fit_params = {'n_iter': 25, 'cv_iter': skf, 'n_cores': 1, 'optimizer': None}

optimizer_params = {'ranker_class': GeneralObjectRanker, 'fit_params': fit_params, 'ranker_params': ranker_params,
                        'random_state': random_state}
X, Y = make_intransitive_medoids(**medoids_params)
optimizer_model = ParameterOptimizer(**optimizer_params)
optimizer_model.fit(X, Y, **optimizer_fit_params)

In [ ]:
print(list(skf.split(Y)))
tau = []
for train_indices, test_indices in list(skf.split(Y)):
    train = X[train_indices]
    train_orderings = Y[train_indices]
    test = X[test_indices]
    test_orderings = Y[test_indices]
    iblr = GeneralObjectRanker(n_object_features=n_features, n_objects=n_objects)
    iblr.fit(train, train_orderings, epochs=1000, verbose=1)
    predicted = iblr(test)
    tau.append(kendalls_mean(predicted, test_orderings))
tau = np.array(tau)
print(tau)
print(np.mean(tau))
print(np.std(tau))

[(array([ 182,  463, 8652, ..., 3768, 7485, 5063]), array([8753, 9734, 5707, ..., 4359, 3257, 8149]))]


/home/pritha/git/pref_learning_neural_network/bordarank/layers.py:119: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ad...)`
  output=self.feature_repr)


Train on 4500 samples, validate on 500 samples
Epoch 1/10000
4500/4500 [==============================] - 2s - loss: 8.5493 - zero_one_rank_loss_for_scores_ties: 0.3882 - val_loss: 8.1146 - val_zero_one_rank_loss_for_scores_ties: 0.2764